In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

Cargamos el dataframe desde el archivo .csv

In [2]:
df = pd.read_csv('datasets\\07-emigracion.csv')
df.head()

,Sexo,Mes,Año,Número emigrantes
0,Hombre,Enero,2009,1716
1,Hombre,Febrero,2009,1950
2,Hombre,Marzo,2009,2515
3,Hombre,Abril,2009,1996
4,Hombre,Mayo,2009,1911


Dado que las fechas están separadas en dos columnas vamos a convertir los nombres de los meses al número que les corresponde, para esto vamos a usar un diccionario y una función lambda para hacerlos fecha

In [ ]:
meses = {'Enero'     : 1, 'Febrero':  2, 'Marzo'    :  3, 'Abril'    :  4,
         'Mayo'      : 5, 'Junio'  :  6, 'Julio'    :  7, 'Agosto'   :  8,
         'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12
        }

In [ ]:
df['fecha'] = df.apply(lambda x: pd.to_datetime('{}-{:02d}-01'.format(x.Año, meses[x.Mes])), axis=1)
df.head()

No vamos a usar las columnas de mes y año, así que las podemos eliminar

In [ ]:
df.drop(['Mes', 'Año'], axis=1, inplace=True)

Cambiamos el nombre de las columnas para que sea más fácil usarlas

In [ ]:
df.columns = ['sexo', 'n', 'fecha']
df.head()

Como vamos a hacer uso de los hombres y mujeres por separado, vamos a crear dos dataframes, uno para cada uno, a partir de la condición de su sexo

In [ ]:
hom = df[df.sexo == 'Hombre']
muj = df[df.sexo == 'Mujer']

In [ ]:
plt.figure(figsize=(12, 9))
plt.style.use('bmh')

plt.plot(hom.fecha, hom.n, 'b', label='hombres')
plt.plot(muj.fecha, muj.n, 'r', label='mujeres')
plt.xticks(rotation=90)

plt.legend(loc='best')
plt.ylabel('# de Personas')
plt.ylabel('Fecha')
plt.title('Personas migrantes en España')

## Regresiones (líneas de tendencia)

No existe una forma directa de pintar las líneas de tendencia, por lo que tenemos que calcular el valor de la propia línea

Y nos vamos a encontrar con un problema... La función de numpy para ajustar (regresiones lineales o polinomiales) no soporta variables del tipo `timestamp`

In [ ]:
plt.figure(figsize=(12, 9))
plt.style.use('bmh')

plt.plot(hom.fecha, hom.n, 'b', label='hombres')
plt.plot(muj.fecha, muj.n, 'r', label='mujeres')
plt.xticks(rotation=90)

# Regresión lineal para los hombres
z = np.polyfit(hom.fecha, hom.n, 1)
p = np.poly1d(z)
plt.plot(hom.fecha, p(hom.fecha), c="b", ls=':')

plt.legend(loc='best')
plt.ylabel('# de Personas')
plt.ylabel('Fecha')
plt.title('Personas migrantes en España')

Por lo que tenemos que crear nuevos valores para las X

La siguiente instrucción creará un rango de 0 hasta el número de fechas que haya para los hombres (y las mujeres)

In [ ]:
x = range(0, len(hom))

In [ ]:
x = range(0, len(hom.fecha))

plt.figure(figsize=(12, 9))
plt.style.use('bmh')

plt.plot(x, hom.n, 'b', label='hombres')
plt.plot(x, muj.n, 'r', label='mujeres')

Ahora tenemos el problema de que en el eje de las X no aparecen las fechas, sino los valores del rango, vamos a añadir las etiquetas a cada valor de las X con `plt.ticks()`

In [ ]:
x = range(0, len(hom.fecha))

plt.figure(figsize=(12, 9))
plt.style.use('bmh')

plt.plot(x, hom.n, 'b', label='hombres')
plt.plot(x, muj.n, 'r', label='mujeres')

# Los xtickers (xt), van a ir desde cero, hasta el número de renglones que haya en hom, en
# intervalos de 6 (seis meses)
#
xt = hom.fecha[range(0, len(hom), 6)]

# Con xticks() indicamos cuáles son las etiquetas que se van a poner, primero decimos donde
# y después decimos qué
#
# El apply lo utilizamos para que solo se vea la fecha sin la hora, que es el default para
# este tipo de variables
#
plt.xticks(range(0, len(hom.fecha), 6), 
           xt.apply(lambda x: x.strftime('%Y-%m-%d')), 
           rotation=90)

# Finalmente calculamos las regresiones lineales usando la función polyfit() de Numpy
# El primer parámetro es el valor de las X (el rango que declaramos)
# El segundo es el valor de la variable dependiente (el número de personas que migraron
#    por mes)
# El tercero es el grado de la función 1 es lineal, 2 cuadrática, 3 cúbica, etc.
#
z = np.polyfit(x, hom.n, 1)
p = np.poly1d(z)
plt.plot(x, p(x), c="b", ls=':')

z = np.polyfit(x, muj.n, 1)
p = np.poly1d(z)
plt.plot(x, p(x), c="r", ls=':')

plt.legend(loc='best')
plt.ylabel('# de Personas')
plt.ylabel('Fecha')
plt.title('Personas migrantes en España')

Una vez que vemos las dos regresiones lineales, vemos como el ritmo de migración de las mujeres es mayor que el de los hombres, y de hecho se cruza alrededor de enero de 2012, quizás influenciado porque en 2011 hubo más mujeres migrando y fue el año en que más migración hubo.

A continuación, se presenta la misma gráfica, pero con **seaborn**, para obtener una gráfica más agradable, sin embargo, las librerías de Seaborn tienen el mismo problema que las de matplotlib, que es que no existe una forma de obtener una regresión lineal de forma automática, y los métodos de Seaborn requieren un tratamiento similar a lo que se hizo arriba para matplotlib, sin embargo, se puede mezclar una y otra librería, por lo que, la gráfica principal será con Seaborn y las líneas de regresión con matplotlib

In [ ]:
x = range(0, len(hom.fecha))

plt.figure(figsize=(12, 9))
chart = sns.lineplot(x='fecha', y='n', data=df, 
                     hue='sexo', markers=True, 
                     style='sexo', dashes=False)
chart.set(title='Personas Migrantes', ylabel='# Personas', xlabel="Fecha")


# Regresión lineal para los hombres
z = np.polyfit(x, hom.n, 1)
p = np.poly1d(z)
plt.plot(hom.fecha, p(x), c="b", ls=":")

z = np.polyfit(x, muj.n, 1)
p = np.poly1d(z)
plt.plot(hom.fecha, p(x), c="r", ls=':')


### Una pirámide poblacional

Esta es una gráfica que me gusta mucho para analizar dos variables, comunmente se usa como pirámide poblacional y es realmente fácil de hacer, cuando se conoce el truco.

En realidad se trata de dos barras, pero en una, los valores se han hecho negativos, para que se pinten del lado izquierdo del eje central y los otros positivos, para que estén del lado derecho

In [ ]:
plt.figure(figsize=(12,10))
plt.style.use('bmh')

# La primera, la barra de los hombres, a la derecha, es una barra horizontal normal
#
plt.barh(hom.fecha, hom.n, color='teal', label='Hombre', height=25)

# La segunda es igual que la primera, pero se multiplica por -1 para que caigan los
# valores del lado izquierdo
#
plt.barh(muj.fecha, muj.n * -1, color='pink', label="Mujer", height=25)

# Con esta instrucción, ponemos los títulos en el eje de las x, el primer parámetro
# dice dónde queremos poner las etiquetas y el segundo dice las etiquetas que vamos
# a poner, así "engañamos" a quien la ve, poniendo solo números positivos
#
plt.xticks([-4000, -3000, -2000, -1000, -500, 0, 500, 1000, 2000, 3000, 4000], 
           [4000, 3000, 2000, 1000, 500, 0, -500, 1000, 2000, 3000, 4000], rotation=90)

# Para tener simetría, fijamos los límites apenas por fuera del rango de los datos
#
plt.xlim(-4500, 4500)

plt.legend(loc='best')
plt.title('Migración por Género')
plt.xlabel('# de Personas')
plt.ylabel('Fecha')

### Por semestre

Vamos a repetir la gráfica superior, pero ahora, en vez de dibujar cada mes, lo haremos con cada semestre

Una forma sencilla es añadir una columna al dataframe principal que nos indique en qué semestre está la observación

[Documentación de Pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.month.html)

In [ ]:
df['semestre'] = (df['fecha'].dt.month-1) // 6
df.head(15)

Ahora que tenemos los semestres, podemos hacer agrupaciones usando la función `group_by()`

In [ ]:
df.groupby(["semestre", "sexo"])["n"].sum()

Ya vimos los datos agrupados por semestre, pero omitimos el año 🤦‍♂️

Fácilmente lo podemos añadir a otra columna, obteniendo el año de la fecha y juntándolo con el semestre en un string

In [ ]:
df['anosem'] = df.apply(lambda x: '{}-{}'.format(x.fecha.year, x.semestre), axis=1)
df.head(15)

In [ ]:
df.groupby(["anosem", "sexo"])["n"].sum()

Con este nuevo dataframe, podemos hacer la gráfica que buscábamos... ¿cuál es la migración por sexo por cada semestre?

Por comodidad, voy a crear dos nuevos dataframes, uno por sexo

Pero antes de hacer eso, es necesario crear un dataframe a partir de las agrupaciones

In [ ]:
sems = df.groupby(["anosem", "sexo"])["n"].sum().to_frame().reset_index()
sems

In [ ]:
homs = sems[sems.sexo == 'Hombre']
mujs = sems[sems.sexo == 'Mujer']
homs.head()

Para que se despliegue bien las gráficas, requerimos saber los valores máximos de cada caso, para ponerlo como los límites de la gráfica

In [ ]:
print('Máxima migración para los hombres: {:,}'.format(homs.n.max()))
print('Máxima migración para las mujeres: {:,}'.format(mujs.n.max()))

In [ ]:
plt.figure(figsize=(12,10))
plt.style.use('bmh')

# La primera, la barra de los hombres, a la derecha, es una barra horizontal normal
#
plt.barh(homs.anosem, homs.n, color='teal', label='Hombre')

# La segunda es igual que la primera, pero se multiplica por -1 para que caigan los
# valores del lado izquierdo
#
plt.barh(mujs.anosem, mujs.n * -1, color='pink', label="Mujer")

# Con esta instrucción, ponemos los títulos en el eje de las x, el primer parámetro
# dice dónde queremos poner las etiquetas y el segundo dice las etiquetas que vamos
# a poner, así "engañamos" a quien la ve, poniendo solo números positivos
#
plt.xticks([  -20000,    -10000,    -5000,   0,    5000,    10000,    20000], 
           ['20,000',  '10,000',  '5,000', '0', '5,000', '10,000', '20,000'], rotation=90)

# Para tener simetría, fijamos los límites apenas por fuera del rango de los datos
#
plt.xlim(-21500, 21500)

plt.legend(loc='best')
plt.title('Migración por Género')
plt.xlabel('# de Personas')
plt.ylabel('Fecha')

In [ ]:
x = range(0, len(homs))

plt.figure(figsize=(12, 9))
chart = sns.lineplot(x='anosem', y='n', data=sems, 
                     hue='sexo', markers=True, 
                     style='sexo', dashes=False)
chart.set(title='Personas Migrantes', ylabel='# Personas', xlabel="Fecha")


# Regresión lineal para los hombres
z = np.polyfit(x, homs.n, 1)
p = np.poly1d(z)
plt.plot(homs.anosem, p(x), c="b", ls=":")

z = np.polyfit(x, mujs.n, 1)
p = np.poly1d(z)
plt.plot(homs.anosem, p(x), c="r", ls=':')